In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
n_points = '5k'

if n_points == '5k':
    target_filename = 'target_mesh.vtk'
    source_filename = 'source_mesh.vtk'
elif n_points == '15k':
    target_filename = 'target_mesh_15k.vtk'
    source_filename = 'source_mesh_15k.vtk'
    
target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

Best results yet with: 

```
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_ordering=10000)
```

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([5,5]),
                      n_coords_spectral_ordering=10000,
                      n_coords_spectral_registration=1000)

Loaded Mesh 1
Laplacian Type:   (0, 116)	-0.6389734303490815
  (0, 113)	-2.678118745579795
  (0, 109)	-1.2500425471012213
  (0, 108)	-1.9045441334813615
  (0, 4)	-1.7641652514381616
  (0, 1)	-0.9062419260566129
  (0, 0)	9.142086034006233
  (1, 112)	-0.9529889372755104
  (1, 109)	-1.6263610371724582
  (1, 7)	-1.8753105319717038
  (1, 4)	-1.2892813691845673
  (1, 2)	-2.0834818963119495
  (1, 1)	8.69549826785341
  (1, 0)	-0.8680744959372211
  (2, 112)	-0.942114215298563
  (2, 9)	-1.0747810066097707
  (2, 7)	-1.4252743568554436
  (2, 5)	-1.6567766344275379
  (2, 3)	-1.2639207918112159
  (2, 2)	8.70189222410883
  (2, 1)	-2.3390252191062983
  (3, 114)	-0.9386401035837287
  (3, 112)	-0.5630780933218489
  (3, 111)	-2.464062420114335
  (3, 6)	-1.6069666910682094
  :	:
  (4996, 4992)	-1.921053165402161
  (4996, 4983)	-1.3977236859914393
  (4996, 4979)	-0.9026579548767935
  (4996, 4976)	-1.6131113417338356
  (4997, 4999)	-0.5272633707379142
  (4997, 4998)	-1.2887753889579954
  (4997, 4997)	8.2988

### View mesh features

In [4]:
reg.graph_source.eig_vals

array([0.00597647, 0.00783398, 0.01074402, 0.01826065, 0.02331632,
       0.0271262 ])

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [5]:
eig_vec = 0

In [6]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [7]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second it will print the rigid registration progress and ultimately the registration "results"/"parameters"
- Third it will print the same for the non-rigid registration progress. 


In [8]:
reg.align_maps()

Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  -0    
     1	|  -1    
     2	|  2     
     3	|  -3    
     4	|  -4    
     5	|  5     
*Negative source values means those eigenvectors were flipped*

Appending Extra Features to Spectral Coords
Number of features (including spectral) used for registartion: 8

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML:-15274.494; 	ML change (error): 15274.494; 	Sigma^2:     0.015; 	Sigma^2 change:     0.060
[                                                                        ]
Iteration:2
ML:-19938.298; 	ML change (error):  4663.804; 	Sigma^2:     0.006; 	Sigma^2 change:     0.009
[                                                                        ]
Iteration:3
ML:-22624.507; 	ML change (error):  2686.210; 	Sigma^2:     0.002; 	Sigma^2 change:     0.004
[                                                                        ]
Iteration:4
ML:-25786.539; 	ML change (err

Iteration:46
ML:-32436.946; 	ML change (error):     0.941; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:47
ML:-32437.643; 	ML change (error):     0.697; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:48
ML:-32438.693; 	ML change (error):     1.050; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:49
ML:-32441.238; 	ML change (error):     2.545; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:50
ML:-32443.428; 	ML change (error):     2.190; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[============                                                            ]
Iteration:51
ML:-32444.611; 	ML change (error):     1.183; 	Sigma^2:     0.000; 	Sigma^2 c

Iteration:98
ML:-32461.635; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:99
ML:-32461.635; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:100
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:101
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:102
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:103
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma

Iteration:144
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==================================                                      ]
Iteration:145
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==================================                                      ]
Iteration:146
ML:-32461.635; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:147
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:148
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:149
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:190
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=============================================                           ]
Iteration:191
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=============================================                           ]
Iteration:192
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==============================================                          ]
Iteration:193
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==============================================                          ]
Iteration:194
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==============================================                          ]
Iteration:195
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:240
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=========================================================               ]
Iteration:241
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=========================================================               ]
Iteration:242
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[==========================================================              ]
Iteration:243
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========================================================              ]
Iteration:244
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========================================================              ]
Iteration:245
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:285
ML:-32461.635; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:286
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:287
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:288
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=====================================================================   ]
Iteration:289
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[=====================================================================   ]
Iteration:290
ML:-32461.636; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

array([   2,    7,    2, ..., 4979, 4983, 4982])

## View the spectral coordinates

In [9]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0, include_rigid_aligned=True, include_unaligned=True)

Viewer(point_set_colors=array([[0.8392157 , 0.        , 0.        ],
       [0.54901963, 0.23137255, 1.       …

In [10]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [11]:
from itkwidgets import view

In [12]:
view(point_sets=[reg.smoothed_target_coords, reg.source_projected_on_target])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [13]:
view(point_sets=[reg.source_projected_on_target])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [14]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

3542

In [15]:
reg.get_initial_correspondences()

In [16]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

4120

In [17]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…